In [ ]:
!pip install streamlit
!pip install pyngrok==4.1.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 164.8/164.8 kB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.7/4.7 MB 65.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/82.1 kB 8.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 kB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 kB 7.1 MB/s eta 0:00:00
  Created wheel for validators: filename=validators-0.20.0-py3-none-any.whl size=19579 sha256=23ec776aecd124578f819fc1299dad3dc297eb8699385a6c520b585c76333ed0
  Stored in directory: /root/.cache/pip/wheels/2d/f0/a8/1094fca7a7e5d0d12ff56e0c64675d72aa5cc81a5fc200e849
Successfully built validators
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/

In [ ]:
from pyngrok import ngrok

# public_url = ngrok.connect(port='8501')
# print(public_url)

In [ ]:
!pip install Pillow

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
%%writefile app.py 
from fingerprintverification import *
import streamlit as st
import sqlite3
import uuid
import base64
import cv2
import numpy as np
from PIL import Image
import io
from io import BytesIO

# Connect to the SQLite database
conn = sqlite3.connect('/content/drive/MyDrive/patients.db')
med = sqlite3.connect('medical_record.db')

# Define a function to create the patients table in the database
def create_ptable():
    c = conn.cursor()
    
    #c.execute('''ALTER TABLE patients ADD COLUMN patient_id INTEGER NULL IF NOT EXISTS''')
    conn.commit()
    
   
    c.close()


# Define a function to add a new patient to the database
def add_patient(name, bod, gender, mobile_number, address,Vaccination,image_files):
    c = conn.cursor()
    patient_id = str(uuid.uuid4())
    c.execute("INSERT INTO patients (patient_id, name,birthdate, gender, mobile_number, address, Vaccination) VALUES (?, ?, ?, ?, ?, ?, ?)",
              (patient_id, name, bod, gender,mobile_number, address, Vaccination))
    image = None
    print("Image Files is ", image_files)
    #file_bytes = np.asarray(bytearray(image_files.read()), dtype=np.uint8)
    #image = cv2.imdecode(file_bytes, 1)
    for uploaded_file in image_files:
        # Read the uploaded image
        file_bytes = np.asarray(bytearray(uploaded_file.read()), dtype=np.uint8)
        image = cv2.imdecode(file_bytes, 1)
    c.execute(f"UPDATE patients SET Left_thumb_finger = ? WHERE patient_id = ?", (image, patient_id))
    conn.commit()
    c.close()
    return patient_id


# Define a function to retrieve a patient from the database by ID
def get_patient_by_id(patient_id):
    c = conn.cursor()
    c.execute("SELECT * FROM patients WHERE patient_id=?", (patient_id,))
    m = med.cursor()
    patient = c.fetchone()
    if patient:
        # get medical records associated with patient's medical ID

        c.close()
        return patient
    else:
        c.close()
        return None, None



# c = conn.cursor()
# c.execute("SELECT image FROM patients")
# rows = c.fetchall()
# image = [row[0] for row in rows]
# images = os.listdir(image)

#name, bod, gender, mobile_number, address,Vaccination,image_files
def display_patient(patient):
    st.write('**Name:**', patient[2])
    st.write('**Contact:**', patient[5])
    st.write('**Address:**', patient[6])
    st.write('**Patient ID:**', patient[1]) 
    st.write('**Gender:**', patient[4]) 
    st.write('**Birth Date:**', patient[3])
    st.write('**Vaccination:**', patient[7])             

def enroll_patient() :
    # Define input fields for patient details
    name = st.text_input('Name')
    age = st.number_input('Age', min_value=0, max_value=150, value=0, step=1)
    mobile_number = st.text_input('Contact')
    address = st.text_input('Address')
    bod = st.text_input('Birth Date')
    Vaccination = st.text_input('Vaccination Records')
    gender = st.text_input('Gender')
    

    # Define input fields for patient images
    image_files = st.file_uploader('Upload image', type=['jpg', 'jpeg', 'png', 'bmp'], accept_multiple_files=True)


    # Define a button to submit the patient data
    if st.button('Enroll'):
        if name and mobile_number and address and image_files:
            patient_id = add_patient(name, bod, gender, mobile_number, address,Vaccination,image_files)
            st.success(f'Patient enrolled with ID: {patient_id}')
        else:
            st.warning('Please fill out all required fields and upload at least one image.')

def get_image_path(patient_id):
    conn = sqlite3.connect('/content/drive/MyDrive/patients.db')
    c = conn.cursor()

    c.execute("SELECT Left_thumb_finger FROM patients WHERE patient_id=?", (patient_id,))
    result = c.fetchone()[0]
    print(result)
    #st.write(result)
    c.close()
   # image_data = patient[5]
    #if image_data is not None:
     #       image_bytes = base64.b64decode(image_data)

    #nparr = np.fromstring(result, np.uint8)
    #img_np = cv2.imdecode(nparr, cv2.CV_LOAD_IMAGE_COLOR)
    return result

def comparison(image,valid_minutiae1,local_structures1,ref_cell_coords1,test_image,valid_minutiae,local_structures,ref_cell_coords):
  f1, m1, ls1 = test_image, valid_minutiae, local_structures
  f2, m2, ls2 = image, valid_minutiae1, local_structures1
  ofn = test_image # Fingerprint of the same finger
  dists = np.sqrt(np.sum((ls1[:,np.newaxis,:] - ls2)**2, -1))
  dists /= (np.sqrt(np.sum(ls1**2, 1))[:,np.newaxis] + np.sqrt(np.sum(ls2**2, 1))) # Normalize as in eq. (17) of MCC paper
  # Select the num_p pairs with the smallest distances (LSS technique)
  num_p = 5 # For simplicity: a fixed number of pairs
  pairs = np.unravel_index(np.argpartition(dists, num_p, None)[:num_p], dists.shape)
  score = 1 - np.mean(dists[pairs[0], pairs[1]]) # See eq. (23) in MCC paper
  print(f'Comparison score: {score:.2f}')

  return score  

def verifi(image_1,image_2):
  if image_1 is not None and image_2 is not None:
    print("hola")
    # If two images were uploaded, read them into numpy arrays
    #image_1 = cv2.imdecode(np.frombuffer(img1.read(), np.uint8), cv2.IMREAD_GRAYSCALE)
    #image_2 = cv2.imdecode(np.frombuffer(image_2, np.uint8), cv2.IMREAD_GRAYSCALE)
    print(type(image_1))
    print(type(image_2))
    #image_1 = Image.fromarray(image_1)
    #image_2 = Image.fromarray(image_2)
    # Resize images to the same size (assuming both images have the same resolution)
    height, width = image_1.shape[:2]
    image_2_resized = cv2.resize(image_2, (width, height))
    
    image_1 = cv2.cvtColor(image_1, cv2.COLOR_BGR2GRAY)
    #image_2 = cv2.cvtColor(image_2, cv2.COLOR_BGR2GRAY)
    # Calculate the mean squared error (MSE) between the two images
    mse = ((image_1 - image_2_resized) ** 2).mean()
    print("hola2")
    # Display the MSE and whether the images match
    print(mse)
    #st.write("MSE:", mse)
    if mse < 70:
        return 1
    else:
        return 0

def verify(test,img) :
    mask1, gx21, gy21, gx1, gy1 = segmentation(test)
    orientations1 = local_ridge_orientation(gx21, gy21, gx1, gy1)
    ridge_period1 = local_ridge_frequency(test)
    enhanced1 = fingerprint_enhancement(ridge_period1,test,orientations1,mask1)
    all_8_neighborhoods1,filtered_minutiae1,cn_values1, cn1 = detect_minutiae(enhanced1,mask1)
    valid_minutiae1 = minutiae_directions(all_8_neighborhoods1,filtered_minutiae1,cn_values1,cn1)
    local_structures1,ref_cell_coords1 = local_structures_calc(valid_minutiae1)

    mask, gx2, gy2, gx, gy = segmentation(img)
    orientations = local_ridge_orientation(gx2, gy2, gx, gy)
    ridge_period = local_ridge_frequency(img)
    enhanced = fingerprint_enhancement(ridge_period,img,orientations,mask)
    all_8_neighborhoods,filtered_minutiae,cn_values, cn = detect_minutiae(enhanced,mask)
    valid_minutiae = minutiae_directions(all_8_neighborhoods,filtered_minutiae,cn_values,cn)
    local_structures,ref_cell_coords = local_structures_calc(valid_minutiae)

    match_score = comparison(img,valid_minutiae,local_structures,ref_cell_coords,test,valid_minutiae1,local_structures1,ref_cell_coords1)
    print(match_score)
    if match_score > 0.8 :
      return 1
    else :
      return 0


def view_patient_info() :
    patient_id = st.text_input('Patient ID')
    patient = get_patient_by_id(patient_id)
    test = st.file_uploader('Upload image', type=['jpg', 'jpeg', 'png', 'bmp'], accept_multiple_files=True)
    #print(type(test[0])) 
    file_bytes = np.asarray(bytearray(test[0].read()), dtype=np.uint8)
    test = cv2.imdecode(file_bytes, 1)
    print(type(test))
    #print(patient_id)
    image_data = get_image_path(patient_id)
    #print(image_data)
    #print(type(image_data))
    file_bytes = np.frombuffer(image_data, np.uint8)
    #file_bytes = np.asarray(bytearray(image_data.read()), dtype=np.uint8)
    #print(file_bytes)
    #img = cv2.imdecode(file_bytes, 1)
    #img = cv2.imdecode(np.frombuffer(image_data, np.uint8), -1)
    print(type(file_bytes))
    #img = Image.open(BytesIO(image_data))
    #data = dict()
    res = verifi(test,file_bytes)

    if res == 1:
      print(res)
      st.subheader('Patient Details')
      display_patient(patient)
    else :
      st.write('Verification Failed')

def app():
    st.title('Patient Enrollment System')
    #st.set_page_config(page_title='Patient Enrollment System')
    menu = ['Enroll Patient', 'View Patient Information']
    choice = st.sidebar.selectbox('Select an option', menu)
    if choice == 'Enroll Patient':
        enroll_patient()
    elif choice == 'View Patient Information':
        view_patient_info()


if __name__ == '__main__':
    app()

Overwriting app.py


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import sqlite3
conn = sqlite3.connect('/content/drive/MyDrive/patients.db')
c = conn.cursor()



result = c.execute("SELECT * FROM patients").fetchall()
print(result)
# Create a DataFrame from the query result
df_result = pd.DataFrame(result, columns=[description[0] for description in c.description])

# Display the DataFrame
df_result

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



,id,patient,Name,birthdate,gender,mobile_number,address,Vaccination,Left_little_finger,Left_middle_finger,Left_index_finger,Left_ring_finger,Left_thumb_finger,Right_little_finger,Right_middle_finger,Right_index_finger,Right_ring_finger,Right_thumb_finger,patient_id
0,1,4ee2c837-e60f-4c54-9fdf-8686bc70760b,Rosamaria Pfannerstill,1929-04-08,F,999-78-5976,18797 Karson Burgs Suite 444 Palmer Town MA 01...,Influenza seasonal injectable preservative ...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,None
1,2,efaf74f9-3de3-45dd-a5d5-26d08e8a3190,Loan Bashirian,2016-12-15,F,999-59-9186,301 Eula Radial Suite 298 Brockton MA 02305 US,Influenza seasonal injectable preservative ...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,None
2,3,aaa4c718-2f48-4c13-9ad0-d287cf280824,Angelo Buckridge,1943-11-28,M,999-43-3780,8693 Fred Crossroad New Bedford MA 02746 US,Influenza seasonal injectable preservative ...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,None
3,4,a1851c06-804e-4f31-9d8f-388cd52d4ad0,Cami Terry,1954-10-22,F,999-53-5542,344 Olson Road Apt. 936 Attleboro MA 02703 US,Td (adult) preservative free,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,None
4,5,48074b70-4db4-4ab0-b9e8-361bd2ba6216,Giovanni Russel,1935-04-08,M,999-34-8549,5780 Corwin Trafficway Dartmouth MA 02714 US,Influenza seasonal injectable preservative ...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,b'BM\n\x9b\x00\x00\x00\x00\x00\x00\x8a\x00\x00...,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
602,603,None,sdfg,None,None,1234,asdf,sdfgh,None,None,None,None,b'\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0...,None,None,None,None,None,5cebf15c-e949-400f-9d31-9216204e0f1b
603,604,None,sdf,None,None,1234,sdfg,sdfg,None,None,None,None,b'\xa0\xa0\xa0\xa0\xa0\xa0\xa0

In [ ]:
!streamlit run app.py & npx localtunnel --port 8501